<a href="https://colab.research.google.com/github/ved1beta/kaggle_pr_compition/blob/main/mainmain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
df.isna().sum()


In [ ]:
modes = df.mode().iloc[0]
modes

In [ ]:
df.fillna(modes, inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
import numpy as np

df.describe(include=(np.number))

In [ ]:
df['LogFare']=np.log1p(df['Fare']);

In [ ]:
df['LogFare'].hist()

In [ ]:
df.describe(include=[object])

In [ ]:
df = pd.get_dummies(df, columns=['Sex','Embarked','Pclass'], dtype=int)

In [ ]:
df

In [ ]:
added_cols = ['Sex_male', 'Sex_female', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
df[added_cols].head()

In [ ]:
from torch import tensor

t_dep = tensor(df.Survived)



In [ ]:
import torch
indep_cols = ['Age', 'SibSp', 'Parch', 'LogFare'] + added_cols

t_indep = tensor(df[indep_cols].values, dtype=torch.float)
t_indep

In [ ]:
t_indep.shape

In [ ]:
torch.manual_seed(442)

n_coeffs = t_indep.shape[1]
coeffs = torch.rand(n_coeff)-0.5
coeffs

In [ ]:
t_indep*coeffs

In [ ]:
vals,indices = t_indep.max(dim=0)
t_indep = t_indep / vals

In [ ]:
t_indep*coeffs

In [ ]:
preds = (t_indep*coeffs).sum(axis=1)


In [ ]:
preds[:10]

In [ ]:
loss = torch.abs(preds-t_dep).mean()
loss

In [ ]:
def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

In [ ]:
coeffs.requires_grad_()

In [ ]:
loss = calc_loss(coeffs, t_indep, t_dep)
loss

In [ ]:
dd=coeffs.grad

In [ ]:
dd

In [ ]:
loss = calc_loss(coeffs, t_indep, t_dep)
loss.backward()

In [ ]:
with torch.no_grad():
    coeffs.sub_(coeffs.grad * 0.1)
    print(calc_loss(coeffs, t_indep, t_dep))

# New Section training the modle

In [ ]:
import fastai
from fastai.data.transforms import *
from fastai.data.all import *

In [ ]:
trn_split, val_split= RandomSplitter(seed=42)(df)
trn_split

In [ ]:
trn_indep , val_indep = t_indep[trn_split], t_indep[val_split]
trn_dep , val_dep = t_dep[trn_split], t_dep[val_split]

In [ ]:
def update_coeffs(coeffs, lr=0.1):
        coeffs.sub_(coeffs.grad * lr)

In [ ]:
def one_epoch(coeffs, lr=0.1):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad():
        update_coeffs(coeffs, lr)
        print(f"{loss:.3f}", end=";")

In [ ]:
def one_epoch(coeffs, lr=0.1):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad():
        update_coeffs(coeffs, lr)
        print(f"{loss:.3f}", end=";")

In [ ]:
def init_coeffs():


    return (torch.rand(n_coeffs)-0.5).requires_grad_()

In [ ]:
def train_model(epochs=30, lr=0.01):
    coeffs = init_coeffs()
    for i in range(epochs):
        one_epoch(coeffs, lr)
    return coeffs

In [ ]:
coeffs = train_model(18, 0.02)

In [ ]:
def show_coeffs(): return dict(zip(indep_cols, coeffs.requires_grad_(False)))
show_coeffs()

In [ ]:
preds = calc_preds(coeffs, val_indep)
results = val_dep.bool() == (preds>0.5)
results[:16]

In [ ]:
results.float().mean()


In [ ]:
def acc(coeffs): return (val_dep.bool() == (calc_preds(coeffs, val_indep)>0.5)).float().mean()

using gigmoid :)



In [ ]:
preds

In [ ]:
import sympy

x = sympy.Symbol('x')  # Define 'x' as a SymPy symbol
expr = 1 / (1 + sympy.exp(-x))  # Create the expression using SymPy functions
sympy.plot(expr, (x, -10, 10))

In [ ]:
#redefining cal_preds
def calc_preds(coeffs, indeps): return torch.sigmoid((indeps*coeffs).sum(axis=1))

In [ ]:
coeffs = train_model(lr=0.2)

In [ ]:
acc(coeffs)

In [ ]:
show_coeffs()

In [ ]:
tst_df = pd.read_csv('test.csv')
tst_df['Fare'] = tst_df['Fare'].fillna(tst_df['Fare'].mean())
tst_df['LogFare']=np.log1p(tst_df['Fare']);
tst_df

In [ ]:
tst_df.fillna(modes, inplace=True)
tst_df['LogFare']=np.log1p(tst_df['Fare']);
tst_df = pd.get_dummies(tst_df, columns=['Sex','Embarked','Pclass'], dtype=int)
tst_indep = tensor(tst_df[indep_cols].values, dtype=torch.float)
tst_indep= tst_indep / vals

In [ ]:
tst_df['survived']=(calc_preds(coeffs, tst_indep)>0.5).int()
sub_df=tst_df[['PassengerId','survived']]
sub_df.to_csv('submission.csv',index=False)

In [ ]:
!head submission.csv


In [ ]:
def init_coeffs(n_hidden=20):
    layer1 = (torch.rand(n_coeff, n_hidden)-0.5)/n_hidden
    layer2 = torch.rand(n_hidden, 1)-0.3
    const = torch.rand(1)[0]
    return layer1.requires_grad_(),layer2.requires_grad_(),const.requires_grad_()

In [ ]:
import torch.nn.functional as F

def calc_preds(coeffs, indeps):
    l1,l2,const = coeffs
    res = F.relu(indeps@l1)
    res = res@l2 + const
    return torch.sigmoid(res)

In [ ]:
def update_coeffs(coeffs, lr):
    for layer in coeffs:
        layer.sub_(layer.grad * lr)
        layer.grad.zero_()

In [ ]:
coeffs = train_model(lr=1.4)

In [ ]:
coeffs = train_model(lr=20)

In [ ]:
acc(coeffs)

In [ ]:
def init_coeffs():
    hiddens = [10, 10]  # <-- set this to the size of each hidden layer you want
    sizes = [n_coeff] + hiddens + [1]
    n = len(sizes)
    layers = [(torch.rand(sizes[i], sizes[i+1])-0.3)/sizes[i+1]*4 for i in range(n-1)]
    consts = [(torch.rand(1)[0]-0.5)*0.1 for i in range(n-1)]
    for l in layers+consts: l.requires_grad_()
    return layers,consts

In [ ]:
import torch.nn.functional as F

def calc_preds(coeffs, indeps):
    layers,consts = coeffs
    n = len(layers)
    res = indeps
    for i,l in enumerate(layers):
        res = res@l + consts[i]
        if i!=n-1: res = F.relu(res)
    return torch.sigmoid(res)

In [ ]:
def update_coeffs(coeffs, lr):
    layers,consts = coeffs
    for layer in layers+consts:
        layer.sub_(layer.grad * lr)
        layer.grad.zero_()

In [ ]:
coeffs = train_model(lr=4)

In [ ]:
acc(coeffs)